In [1]:
import os
import pandas as pd
import numpy as np
import csv
import time

In [2]:
path_dir = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\data'
excel_file = 'shenghuo_comments_35378.xlsx'
excel_frq_file = 'shenghuo_word_frq_17698.xlsx'

path_file = os.path.join(path_dir,excel_file)
path_frq_file = os.path.join(path_dir,excel_frq_file)

In [3]:
filter_df = pd.read_excel(path_file,usecols=['title','text','user_name','create_time','clean_text','cut_word','pos_word'])
filter_df.index = (filter_df.reset_index()).index + 1

#读取特定个数的高频词，以构建高频词之间的共现关系。
word_frq_filter_df = pd.read_excel(path_frq_file,index_col=0,nrows=238)
    #这里的"nrows=238"，即读取238个词，其数量是根据高频词低频词分界公式计算得出的。

In [4]:
word = list(word_frq_filter_df['word'])

In [7]:
line_list = []
for line in filter_df['cut_word']:
    line_word_list = line.split(' ')
    line_filter_word = []
    for w in line_word_list:
        if len(w) > 1:
            if w in word:
                line_filter_word.append(w)
    if len(line_filter_word) > 1:  #如果此条评论中的词大于1个，则加入列表，否则抛弃。因为如果只有1个，则无法做共现分析。
        line_list.append(line_filter_word)

In [9]:
def co_word_vector(word,line_lists):
    word_vector = np.zeros([len(word),len(word)], dtype='float32')  #float16好像最大为2048，试下float32。float32果然能表示比2048更大的数。
    #word_vector = coo_matrix((len(word),len(word)), dtype=np.int8).toarray()  #此矩阵貌似进行过换算，目前的参数最大是128，并且对某些共现对降过权。还要试验、研究以确认。
    print(f"矩阵形状为：{word_vector.shape}")
    print(f"不重复词数为：{len(word)}")
    print(f"评论总数为：{len(line_lists)}")

    m = 1
    for nums in line_lists:

        #循环遍历关键词所在位置 设置word_vector计数
        i = 0
        j = 0
        while i<len(nums):         #ABCD共现 AB AC AD BC BD CD加1
            #第一次循环则句子里第一个词和第二个词。第二次循环则句子里第二个词和第三个词，依次类推。
            j = i + 1  #i为第一个词，j则为第二个词。
            w1 = nums[i]           #w1为句子里第一个词
            while j<len(nums):
                w2 = nums[j]       #w2为句子里第二个词
                #从word数组中找到单词对应的下标
                k = 0
                n1 = 0
                while k<len(word): #循环遍历word列表，再将w1与word列表里的词依次比较。
                    if w1==word[k]: #如果单词w1与word列表里的第k个词相同，则k为单词w1在word列表里的下标。
                        n1 = k #n1为单词w1在word列表里的下标。
                        break
                    k = k +1
                #寻找第二个关键字位置
                k = 0
                n2 = 0
                while k<len(word):
                    if w2==word[k]:  #此时k为单词w2在word列表里的下标。
                        n2 = k  #此时n2为单词w2在word列表里的下标。
                        break
                    k = k +1
                #重点: 词频矩阵赋值 只计算上三角
                if n1<=n2:
                    word_vector[n1][n2] = word_vector[n1][n2] + 1
                else:
                    word_vector[n2][n1] = word_vector[n2][n1] + 1
                #print n1, n2, w1, w2
                j = j + 1
            i = i + 1
        if m == 10000 or m == 20000 or m == 30000:
            print(f'已处理{m}条评论，请稍等。')
            print(time.time())
        m += 1
    return word_vector

In [91]:
#将所有词构造成一个去除单字符，且不重复的词列表。
'''
word = []
for line in filter_df['cut_word']:
    line = line.split(' ')
    for w in line:
        if len(w) > 1:
            if w not in word:
                word.append(w)
'''

#定义共现函数。
'''
def co_word_vector(word,line_lists):
    word_vector = np.zeros([len(word),len(word)], dtype='float32')  #float16好像最大为2048，试下float32。float32果然能表示比2048更大的数。
    #word_vector = coo_matrix((len(word),len(word)), dtype=np.int8).toarray()  #此矩阵貌似进行过换算，目前的参数最大是128，并且对某些共现对降过权。还要试验、研究以确认。
    print(f"矩阵形状为：{word_vector.shape}")
    print(f"不重复词数为：{len(word)}")
    print(f"评论总数为：{len(line_lists)}")
    m = 1
    for line in line_lists:

        #去除读取行中的单字符。
        line_list = line.split(' ')
        nums = []
        for w in line_list:
            if len(w) > 1:
                nums.append(w)

        #循环遍历关键词所在位置 设置word_vector计数
        i = 0
        j = 0
        while i<len(nums):         #ABCD共现 AB AC AD BC BD CD加1
            j = i + 1
            w1 = nums[i]           #第一个单词
            while j<len(nums):
                w2 = nums[j]       #第二个单词
                #从word数组中找到单词对应的下标
                k = 0
                n1 = 0
                while k<len(word):
                    if w1==word[k]:
                        n1 = k
                        break
                    k = k +1
                #寻找第二个关键字位置
                k = 0
                n2 = 0
                while k<len(word):
                    if w2==word[k]:
                        n2 = k
                        break
                    k = k +1
                #重点: 词频矩阵赋值 只计算上三角
                if n1<=n2:
                    word_vector[n1][n2] = word_vector[n1][n2] + 1
                else:
                    word_vector[n2][n1] = word_vector[n2][n1] + 1
                #print n1, n2, w1, w2
                j = j + 1
            i = i + 1
        if m == 10000 or m == 20000 or m == 30000:
            print(f'已处理{m}条评论，请稍等。')
            print(time.time())
        m += 1
    return word_vector
'''

'\ndef co_word_vector(word,line_lists):\n    word_vector = np.zeros([len(word),len(word)], dtype=\'float32\')  #float16好像最大为2048，试下float32。float32果然能表示比2048更大的数。\n    #word_vector = coo_matrix((len(word),len(word)), dtype=np.int8).toarray()  #此矩阵貌似进行过换算，目前的参数最大是128，并且对某些共现对降过权。还要试验、研究以确认。\n    print(f"矩阵形状为：{word_vector.shape}")\n    print(f"不重复词数为：{len(word)}")\n    print(f"评论总数为：{len(line_lists)}")\n    m = 1\n    for line in line_lists:\n\n        #去除读取行中的单字符。\n        line_list = line.split(\' \')\n        nums = []\n        for w in line_list:\n            if len(w) > 1:\n                nums.append(w)\n\n        #循环遍历关键词所在位置 设置word_vector计数\n        i = 0\n        j = 0\n        while i<len(nums):         #ABCD共现 AB AC AD BC BD CD加1\n            j = i + 1\n            w1 = nums[i]           #第一个单词\n            while j<len(nums):\n                w2 = nums[j]       #第二个单词\n                #从word数组中找到单词对应的下标\n                k = 0\n                n1 = 0\n                while k<le

In [10]:
def save_co_word_to_txt(word,word_vector,txt_name):
    res = open(path_dir+'\\'+txt_name, "a+", encoding='utf-8')
    i = 0
    while i<len(word):
        w1 = word[i]
        j = 0
        while j<len(word):
            w2 = word[j]
            #判断两个词是否共现 共现&词频不为0的写入文件
            if w1 != w2:  #如果w1与w2不同，即写入记录，否则不操作。
                if word_vector[i][j]>0:
                    #print w1 +" " + w2 + " "+ str(int(word_vector[i][j]))
                    res.write(w1 +" " + w2 + " "+ str(int(word_vector[i][j]))  +  "\n")
            j = j + 1
        i = i + 1
    res.close()

def save_co_word_to_csv(word,word_vector,csv_name):
    c = open(path_dir+'\\'+csv_name,"w", encoding='utf-8', newline='')    #解决空行
    #c.write(codecs.BOM_UTF8)                                 #防止乱码
    writer = csv.writer(c)                                    #写入对象
    writer.writerow(['Word1', 'Word2', 'Weight'])

    i = 0
    while i<len(word):
        w1 = word[i]
        j = 0
        while j<len(word):
            w2 = word[j]
            #判断两个词是否共现 共现词频不为0的写入文件
            if w1 != w2:  #如果w1与w2不同，即写入记录，否则不操作。
                if word_vector[i][j]>0:
                    #写入文件
                    templist = []
                    templist.append(w1)
                    templist.append(w2)
                    templist.append(str(int(word_vector[i][j])))
                    #print templist
                    writer.writerow(templist)
            j = j + 1
        i = i + 1
    c.close()

In [12]:
#经测试，19975个词，32741条评论，整个过程大致在35分钟内完成。
word_vector1 = co_word_vector(word,line_list)
print('共现矩阵构建完毕。')
print(time.time())

save_co_word_to_txt(word,word_vector1,'shenghuo_co_word_1.txt')
print('写入txt文件完毕。')
print(time.time())

save_co_word_to_csv(word,word_vector1,'shenghuo_co_word_1.csv')
print('写入csv文件完毕。')
print(time.time())

矩阵形状为：(238, 238)
不重复词数为：238
评论总数为：30433
已处理10000条评论，请稍等。
1624585675.7602665
已处理20000条评论，请稍等。
1624585678.2774644
已处理30000条评论，请稍等。
1624585680.3176906
共现矩阵构建完毕。
1624585680.4547758
写入txt文件完毕。
1624585680.5744255
写入csv文件完毕。
1624585680.7145114
